#Combined Topic Modeling

In this notebook, we are going to use  **Combined Topic Model** to get the topics out of a collections of articles.

## Topic Models 

Topic models allow you to discover latent topics in your documents in a completely unsupervised way. Just use your documents and get topics out.

## Contextualized Topic Models

What are Contextualized Topic Models? **CTMs** are a family of topic models that combine the expressive power of BERT embeddings with the unsupervised capabilities of topic models to get topics out of documents. 

# **Before you start...**

If you have additional questions about these topics, follow the links:

- you need to work with languages different than English: [click here!](https://contextualized-topic-models.readthedocs.io/en/latest/language.html#language-specific)
- you can't get good results with topic models: [click here!](https://contextualized-topic-models.readthedocs.io/en/latest/faq.html#i-am-getting-very-poor-results-what-can-i-do)
- you want to load your own embeddings: [click here!](https://contextualized-topic-models.readthedocs.io/en/latest/faq.html#can-i-load-my-own-embeddings)


# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing Contextualized Topic Models

First, we install the contextualized topic model library

In [1]:
%%capture
!pip install contextualized-topic-models==2.3.0

In [2]:
%%capture
!pip install pyldavis

In [5]:
!pip uninstall numpy
!pip install numpy==1.22.0

Found existing installation: numpy 1.21.6
Uninstalling numpy-1.21.6:
  Would remove:
    /usr/bin/f2py
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.8
    /usr/local/lib/python3.8/dist-packages/numpy-1.21.6.dist-info/*
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libopenblasp-r0-2d23e62b.3.17.so
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.8/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.21.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.8 MB 27.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have 

## Restart the Notebook

For the changes to take effect, we now need to restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data

We are going to need some data. You should upload a file with one document per line. We assume you haven't run any preprocessing script.

However, if you want to first test the model without uploading your data, you can simply use the test file I'm putting here

In [1]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

In [2]:
!head -n 2 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and


In [3]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt" # EDIT THIS WITH THE FILE YOU UPLOAD

# Importing what we need

In [4]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk

## Preprocessing

Why do we use the **preprocessed text** here? We need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

In [5]:
from nltk.corpus import stopwords as stop_words

nltk.download('stopwords')

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:2000]]

stopwords = list(stop_words.words("english"))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
preprocessed_documents[:2]

['mid peninsula highway across peninsula canadian province ontario although highway connecting fort south decades international study published ministry',
 'died march american photographer photography operated studio silver spring maryland later lived florida magazine photographer year']

We don't discard the non-preprocessed texts, because we are going to use them as input for obtaining the contextualized document representations. 

Let's pass our files with preprocess and unpreprocessed data to our `TopicModelDataPreparation` object. This object takes care of creating the bag-of-words for you and of obtaining the contextualized BERT representations of documents. This operation allows us to create our training dataset.

Note: Here we use the contextualized model "paraphrase-distilroberta-base-v1".


In [7]:
tp = TopicModelDataPreparation("all-mpnet-base-v2")

training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Let's check the first ten words of the vocabulary 

In [8]:
tp.vocab[:10]

['ab',
 'abbreviated',
 'abroad',
 'academic',
 'academy',
 'accepted',
 'access',
 'according',
 'accounting',
 'achieved']

In [11]:
tp.vocab[1990:]

['ye',
 'year',
 'years',
 'yellow',
 'yet',
 'york',
 'young',
 'youth',
 'zealand',
 'zone']

In [9]:
len(tp.vocab)

2000

## Training our Combined TM

Finally, we can fit our new topic model. We will ask the model to find 50 topics in our collection.

In [12]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=10)
ctm.fit(training_dataset) # run the model

Epoch: [10/10]	 Seen Samples: [20000/20000]	Train Loss: 138.32534484863282	Time: 0:00:00.659648: : 10it [00:06,  1.47it/s]
Sampling: [20/20]: : 20it [00:11,  1.79it/s]


# Topics

After training, now it is the time to look at our topics: we can use the 

```
get_topic_lists
```

function to get the topics. It also accepts a parameter that allows you to select how many words you want to see for each topic.

If you look at the topics, you will see that they all make sense and are representative of a collection of documents that comes from Wikipedia (general knowledge). Notice that the topics are in English, because we trained the model on English documents.

In [14]:
ctm.get_topic_lists(5)

[['author', 'international', 'writer', 'french', 'characters'],
 ['released', 'rock', 'directed', 'studio', 'album'],
 ['km', 'lies', 'north', 'south', 'village'],
 ['used', 'information', 'body', 'important', 'often'],
 ['directed', 'film', 'type', 'name', 'described'],
 ['class', 'used', 'designed', 'user', 'three'],
 ['born', 'played', 'player', 'league', 'january'],
 ['area', 'county', 'village', 'district', 'km'],
 ['village', 'district', 'km', 'south', 'east'],
 ['based', 'founded', 'music', 'company', 'federation'],
 ['league', 'born', 'professional', 'football', 'former'],
 ['released', 'album', 'band', 'game', 'published'],
 ['best', 'known', 'professor', 'social', 'actress'],
 ['politician', 'party', 'house', 'member', 'election'],
 ['located', 'town', 'county', 'historic', 'station'],
 ['born', 'player', 'former', 'played', 'american'],
 ['found', 'moth', 'mm', 'family', 'adults'],
 ['family', 'species', 'school', 'mm', 'college'],
 ['located', 'county', 'state', 'river', 'n

# Let's Draw!

We can use PyLDAvis to plot our topic in a nice and friendly manner :)

In [15]:
 lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

Sampling: [10/10]: : 10it [00:05,  1.75it/s]


In [16]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
Sampling: [10/10]: : 10it [00:05,  1.74it/s]
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


# Topic Predictions

Ok now we can take a document and see which topic has been assigned to it. Results will obviously change with respect to the documents you are using. For example, let's predict the topic of the first preprocessed document that is talking about a peninsula.

In [17]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions

Sampling: [5/5]: : 5it [00:02,  1.72it/s]


In [18]:
preprocessed_documents[0] # see the text of our preprocessed document

'mid peninsula highway across peninsula canadian province ontario although highway connecting fort south decades international study published ministry'

In [22]:
len(topics_predictions)

2000

In [23]:
topics_predictions[1]

array([0.05477281, 0.03251416, 0.00971899, 0.00622704, 0.00642256,
       0.02126407, 0.02865335, 0.01243666, 0.02095056, 0.12538103,
       0.02074165, 0.0082305 , 0.18531113, 0.1162115 , 0.01729583,
       0.08087744, 0.02313832, 0.03457896, 0.0891889 , 0.10608453])

In [24]:
import numpy as np
topic_number = np.argmax(topics_predictions[1]) # get the topic id of the first document

In [25]:
topic_number

12

In [26]:
ctm.get_topic_lists(5)[15]

['born', 'player', 'former', 'played', 'american']

In [27]:
ctm.get_topic_lists(5)[topic_number] #and the topic should be about natural location/places/related things

['best', 'known', 'professor', 'social', 'actress']

# Save Our Model for Later Use

In [28]:
ctm.save(models_dir="./")

/usr/local/lib/python3.8/dist-packages/contextualized_topic_models/models/ctm.py:473: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn("This is an experimental feature that we has not been fully tested. Refer to the following issue:"


In [29]:
# let's remove the trained model
del ctm

In [33]:
#CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=10)
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, num_epochs=10, n_components=20)
#/content/contextualized_topic_model_nc_20_tpm_0.0_tpv_0.95_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99
ctm.load("/content/contextualized_topic_model_nc_20_tpm_0.0_tpv_0.95_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                      epoch=9)

/usr/local/lib/python3.8/dist-packages/contextualized_topic_models/models/ctm.py:498: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn("This is an experimental feature that we has not been fully tested. Refer to the following issue:"


In [34]:
ctm.get_topic_lists(5)

[['author', 'international', 'writer', 'french', 'characters'],
 ['released', 'rock', 'directed', 'studio', 'album'],
 ['km', 'lies', 'north', 'south', 'village'],
 ['used', 'information', 'body', 'important', 'often'],
 ['directed', 'film', 'type', 'name', 'described'],
 ['class', 'used', 'designed', 'user', 'three'],
 ['born', 'played', 'player', 'league', 'january'],
 ['area', 'county', 'village', 'district', 'km'],
 ['village', 'district', 'km', 'south', 'east'],
 ['based', 'founded', 'music', 'company', 'federation'],
 ['league', 'born', 'professional', 'football', 'former'],
 ['released', 'album', 'band', 'game', 'published'],
 ['best', 'known', 'professor', 'social', 'actress'],
 ['politician', 'party', 'house', 'member', 'election'],
 ['located', 'town', 'county', 'historic', 'station'],
 ['born', 'player', 'former', 'played', 'american'],
 ['found', 'moth', 'mm', 'family', 'adults'],
 ['family', 'species', 'school', 'mm', 'college'],
 ['located', 'county', 'state', 'river', 'n